### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data.loc[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((~data['ThoiGianKetThucLay'].isnull()) ^\
                          (~data['ThoiGianKetThucGiao'].isnull())))]
delivery['LoaiBacklog'] = 'Xử lý giao'
internal_delivery = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery, 'N0'] = delivery.loc[internal_delivery]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery, 'N0'] = delivery.loc[~internal_delivery]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned = data.loc[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (~data['ThoiGianKetThucGiao'].isnull()))]
returned['LoaiBacklog'] = 'Xử lý trả'
internal_return = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return, 'N0'] = returned.loc[internal_return]['ThoiGianKetThucGiao']
returned.loc[~internal_return, 'N0'] = returned.loc[~internal_return]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pick_backlog (backlog lấy)
pickup = data[(data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))]
pickup['LoaiBacklog'] = 'Xử lý lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transit_backlog (backlog luân chuyển)
transit = data.loc[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]
transit['LoaiBacklog'] = 'Xử lý luân chuyển'
transit_filter = (transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh'] == 'Hồ Chí Minh')
transit['N0'] = transit['ThoiGianKetThucLay']
transit.loc[transit_filter, 'N+'] = transit.loc[transit_filter]['N0']+ pd.Timedelta(hours=12)
transit.loc[~transit_filter, 'N+'] = transit.loc[~transit_filter]['N0'] + pd.Timedelta(hours=32)
transit['Aging'] = (datetime.today() - transit['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transit_backlog (backlog luân chuyển trả hàng)
r_transit = data.loc[data['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))]
r_transit['LoaiBacklog'] = 'Xử lý luân chuyển trả'
r_transit_filter = (r_transit['DenTinh'] == 'Hà Nội') | (r_transit['DenTinh'] == 'Hồ Chí Minh')
r_transit['N0'] = r_transit['ThoiGianKetThucGiao']
r_transit.loc[r_transit_filter, 'N+'] = r_transit.loc[r_transit_filter]['N0'] + pd.Timedelta(hours=36)
r_transit.loc[~r_transit_filter, 'N+'] = r_transit.loc[~r_transit_filter]['N0'] + pd.Timedelta(hours=56)
r_transit['Aging'] = (datetime.today() - r_transit['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transit, r_transit])
# convert Aging to hours
data['Aging (hours)'] = data['Aging']/np.timedelta64(1, 'h')
# set e-commerces column to NaN
data['e-commerces'] = np.nan


# name of e-commerces exchange
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# shopee_name['e-commerces'] = 'Shopee'
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# sendo_name['e-commerces'] = 'Sendo'
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# tiki_name['e-commerces'] = 'Tiki'
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'

# lazada_name['e-commerces'] = 'Lazada'
# data['e-commerces'] = shopee_name.append([sendo_name, tiki_name, lazada_name])

# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
# reset the index of dataframe
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,LoaiBacklog,N0,N+,Aging,Aging (hours),e-commerces
0,8FHA6YD7Y,2412466328375296517,2020-08-09 15:15:45,2020-08-10 08:00:00,2020-08-09 21:28:58,NaT,NaT,2020-08-13 23:00:00,NaT,NaN,Huyện Củ Chi,Hồ Chí Minh,E,Thành phố Nha Trang,Khánh Hòa,D,Bưu Cục 242 Tỉnh Lộ 2-Củ Chi-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hồ Chí Minh 02,Lưu kho,6748,19x111x16,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,1,0,0,NaN,1868573,NaN,'Kho Strading Củ Chi',1154025,20GGV48349,2279.0,2275.0,2020-08-20 03:28:46,Xử lý giao,2020-08-20 03:28:46,2020-08-25 03:28:46,-5 days +08:26:55.180910,-111.551339,Shopee
1,8FH46QX1S,5f2f8c0f26068e0001d9f616_90,2020-08-09 12:57:23,2020-08-09 12:57:23,2020-08-11 16:56:06,2020-08-14 08:12:30,NaT,2020-08-14 23:00:00,NaT,NaN,Quận Đống Đa,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 114 Yên Lãng-Q.Đống Đa-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 107 Nguyễn Chí Thanh-Q.Đống Đa-HN,Lưu kho,500,10x10x10,649450,'14/08/2020 13:08:48 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',2 Ngày,1,3,0,NaN,1707220,NaN,'JANNA',1217332,20TWC20687,2329.0,2275.0,2020-08-14 00:25:54,Xử lý giao,2020-08-14 00:25:54,2020-08-19 00:25:54,1 days 11:29:47.180910,35.496439,NaN
2,8FHQQN9RA,NaN,2020-08-09 22:10:57,2020-08-10 08:00:00,2020-08-10 14:47:54,2020-08-14 07:55:18,NaT,2020-08-13 17:54:00,NaT,NaN,Quận Cầu Giấy,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 299 Trung Kính-Q.Cầu Giấy-HN,Lưu kho,500,20x80x15,1236246,'14/08/2020 05:08:32 - GHN-DFC1A7|THAY ĐỔI THÔ...,'2 thảm loang đỏ vht',2 Ngày,1,1,0,NaN,247756,NaN,'Hương Bi',1113334,20QAH32120,2268.0,2275.0,2020-08-14 00:01:45,Xử lý giao,2020-08-14 00:01:45,2020-08-19 00:01:45,1 days 11:53:56.180910,35.898939,NaN
3,8FHX6KL4K,S1769147O317,2020-08-09 11:03:56,2020-08-09 11:03:56,2020-08-09 11:57:02,2020-08-11 09:35:03,NaT,2020-08-12 16:58:00,NaT,NaN,Quận Hà Đông,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 23 Ngõ 164 Ỷ La-Q.Hà Đông-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 23 Ngõ 164 Ỷ La-Q.Hà Đông-HN,Đang giao hàng,10,10x5x5,1225713,'11/08/2020 07:40:12 - GHN-DFC1A2|KHÔNG LIÊN L...,"'1,2_XL'",2 Ngày,1,8,0,NaN,1718815,NaN,'zee store',1257257,20VCZ54012,2329.0,2275.0,2020-08-10 20:52:28,Xử lý giao,2020-08-10 20:52:28,2020-08-15 20:52:28,4 days 15:03:13.180910,111.053661,NaN
4,8FHAUF4LS,2412288087257088594,2020-08-09 09:23:02,2020-08-09 09:23:02,2020-08-10 19:34:12,NaT,NaT,2020-08-13 10:17:00,NaT,NaN,Quận Gò Vấp,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Bưu Cục 640 Lê Đức Thọ-Q.Gò Vấp-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 640 Lê Đức Thọ-Q.Gò Vấp-HCM,Lưu kho,450,10x10x10,18692,'09/08/2020 08:50:04 - GHN-PFA1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,3,0,0,NaN,205860,NaN,'PRETTY SHOP',1154025,20UYJ93201,2329.0,2275.0,2020-08-11 14:21:37,Xử lý giao,2020-08-11 14:21:37,2020-08-16 14:21:37,3 days 21:34:04.180910,93.567828,Shopee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6095,GHNMP0000453627VNA,GHNMP0000453627VNA,2020-08-17 10:13:34,2020-08-17 10:13:34,2020-08-17 11:20:41,2020-08-18 07:53:52,2020-08-20 07:13:02,2020-08-19 23:00:00,NaT,'GHN-DCD0A2|KHÔNG LIÊN LẠC ĐƯỢC/Khách không ng...,Huyện Mỏ Cày Bắc,Bến Tre,F,Quận 7,Hồ Chí Minh,E,Bưu Cục Mỏ Cày Nam-Bến Tre,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cụ

In [33]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging (hours)',
    'e-commerces',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog' id:181055436>

In [34]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val